In [ ]:
# !pip install gym_super_mario_bros nes_py

In [1]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [2]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

C:\Users\saugi\Documents\randomProjects\venv\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
C:\Users\saugi\Documents\randomProjects\venv\lib\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [3]:
import gym
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import layers, models
from skimage.color import rgb2gray  # Import rgb2gray from the correct module
from skimage.transform import resize

In [4]:
# Define the Deep Q-Network (DQN) model
def build_dqn(input_shape, num_actions):
    model = models.Sequential([
        layers.Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=input_shape),
        layers.Conv2D(64, (4, 4), strides=(2, 2), activation='relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_actions)
    ])
    return model

In [5]:
# Hyperparameters
observation_shape = (84, 84, 1)  # Preprocessed observation shape
num_actions = env.action_space.n
learning_rate = 0.001
discount_factor = 0.99
exploration_max = 1.0
exploration_min = 0.1
exploration_decay = 0.995
batch_size = 32
memory_size = 10000


In [6]:
# Initialize DQN and target DQN
dqn = build_dqn(observation_shape, num_actions)
target_dqn = build_dqn(observation_shape, num_actions)
target_dqn.set_weights(dqn.get_weights())

In [7]:
# Preprocess observation function
def preprocess_observation(observation):
    gray_observation = np.mean(observation, axis=2, keepdims=True)  # Convert to grayscale
    resized_observation = resize(gray_observation, (84, 84))
    return resized_observation


In [8]:
# Optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.Huber()

In [9]:
# Replay memory
replay_memory = []

In [10]:
# Exploration strategy
def exploration_strategy(step):
    return exploration_min + (exploration_max - exploration_min) * np.exp(-exploration_decay * step)


In [11]:
def experience_replay():
    if len(replay_memory) < batch_size:
        return
    
    batch = random.sample(replay_memory, batch_size)
    
    states, actions, rewards, next_states, dones = [], [], [], [], []
    for experience in batch:
        state, action, reward, next_state, done = experience
        preprocessed_state = preprocess_observation(state)
        preprocessed_next_state = preprocess_observation(next_state)
        
        states.append(preprocessed_state)
        actions.append(action)
        rewards.append(reward)
        next_states.append(preprocessed_next_state)
        dones.append(done)
    
    states = np.array(states)
    next_states = np.array(next_states)
    
    q_values = dqn.predict(states)
    q_values_next = target_dqn.predict(next_states)
    
    for i in range(batch_size):
        if dones[i]:
            q_values[i][actions[i]] = rewards[i]
        else:
            q_values[i][actions[i]] = rewards[i] + discount_factor * np.max(q_values_next[i])
    
    with tf.GradientTape() as tape:
        predicted_q_values = dqn(states)
        loss = loss_fn(q_values, predicted_q_values)
    gradients = tape.gradient(loss, dqn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, dqn.trainable_variables))


In [12]:
# Training loop
total_steps = 0
for episode in range(10000):  # You can adjust the number of episodes
    state = env.reset()
    episode_reward = 0
    
    while True:
        exploration_prob = exploration_strategy(total_steps)
        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()
        else:
            q_values = dqn.predict(np.expand_dims(state, axis=0))
            action = np.argmax(q_values)
        
        next_state, reward, done, _, __ = env.step(action)
        next_state = preprocess_observation(next_state)
        
        replay_memory.append((state, action, reward, next_state, done))
        if len(replay_memory) > memory_size:
            replay_memory.pop(0)
        
        state = next_state
        total_steps += 1
        
        if total_steps > batch_size:
            experience_replay()
        
        if total_steps % 100 == 0:
            target_dqn.set_weights(dqn.get_weights())
        
        if done:
            break
    
    print(f"Episode {episode+1}, Reward: {episode_reward}")

C:\Users\saugi\Documents\randomProjects\venv\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
C:\Users\saugi\Documents\randomProjects\venv\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 [==============================] - 0s 24ms/step


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
# env.close()